In [1]:
import argparse
import collections
import functools
import os
import pathlib
import sys
import warnings

warnings.filterwarnings('ignore', '.*box bound precision lowered.*')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# os.environ['MUJOCO_GL'] = 'egl'

import numpy as np
import ruamel.yaml as yaml
import tensorflow as tf
from tensorflow.keras.mixed_precision import experimental as prec

tf.get_logger().setLevel('ERROR')

from tensorflow_probability import distributions as tfd

#sys.path.append(str(pathlib.Path(__file__).parent))

import exploration as expl
import models
import tools
import wrappers


In [2]:
def make_dataset(episodes, config):
    example = episodes[next(iter(episodes.keys()))]
    types = {k: v.dtype for k, v in example.items()}
    shapes = {k: (None,) + v.shape[1:] for k, v in example.items()}
    generator = lambda: tools.sample_episodes(
        episodes, config.batch_length, config.oversample_ends)
    dataset = tf.data.Dataset.from_generator(generator, types, shapes)
    dataset = dataset.batch(config.batch_size, drop_remainder=True)
    dataset = dataset.prefetch(10)
    return dataset

In [3]:
directory="logdir/train_eps"
train_eps = tools.load_episodes(directory, limit=0)
directory="logdir/eval_eps"
eval_eps = tools.load_episodes(directory, limit=1)

In [4]:
train_eps

{'logdir\\train_eps\\20210406T103104-fd68dbcaa2b24ca49b26ff321499265e-4.npz': {'image': array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
  
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
  
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
  
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [28]:
_data={'image':[],'action':[],'reward':[],'discount':[]}

for _,e in train_eps.items():
    for k in e.keys():
        if k=='action':
            _data[k].append(e[k])
            
        
for k in _data.keys():
    print(k)
    _data[k]=tf.ragged.constant(_data[k])

image
action


ValueError: all scalar values must have the same nesting depth

In [18]:
directory="logdir/train_eps"
directory = pathlib.Path(directory).expanduser()
episodes = {}
total = 0
for filename in reversed(sorted(directory.glob('*.npz'))):
    with filename.open('rb') as f:
        episode = np.load(f,allow_pickle=True)
        print(episode)
        episode = {k: episode[k] for k in episode.keys()}
    break
    
episode

{'image': array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
 
        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]],
       dtype=float32),
 'action': array([array([0., 0., 0., 0., 0., 0., 0., 0., 0.]), 8], dtype=object),
 'reward': array([ 0., -1.], dtype=float32),
 'discount': array([1., 0.], dtype=float32)}

In [5]:
class LolArg:
    def __init__(self):
        self.configs=['defaults','prolog']
        self.logdir='logdir'

        
if(False):
    parser = argparse.ArgumentParser()
    parser.add_argument('--configs', nargs='+', required=True)
    parser.add_argument('--logdir', required=True)
    args, remaining = parser.parse_known_args()
else:
    #TODO remove this!!
    args, remaining =LolArg(), []
    #TODO ...
    print(args,remaining)

configs = yaml.safe_load(
  (pathlib.Path() / 'configs.yaml').read_text())
config_ = {}
for name in args.configs:
    config_.update(configs[name])
    parser = argparse.ArgumentParser()
for key, value in config_.items():
    arg_type = tools.args_type(value)
    parser.add_argument(f'--{key}', type=arg_type, default=arg_type(value))


<__main__.LolArg object at 0x0000020897762688> []


In [6]:
config=parser.parse_args(remaining)
train_dataset = make_dataset(train_eps, config)
eval_dataset = iter(make_dataset(eval_eps, config))

In [7]:
train_dataset

<PrefetchDataset shapes: {image: (50, None, 3, 16), action: (50, None), reward: (50, None), discount: (50, None)}, types: {image: tf.float32, action: tf.string, reward: tf.float32, discount: tf.float32}>

In [8]:
_dataset=iter(train_dataset)

In [9]:
next(_dataset)

InternalError: Unsupported object type numpy.int32
	 [[{{node PyFunc}}]]

In [ ]:
x1=tf.RaggedTensor.from_row_lengths(
    values=[3, 1, 4, 1, 5, 9, 2],
    row_lengths=[4, 0, 2, 1])
x2=tf.RaggedTensor.from_row_lengths(
    values=[3, 1, 4, 1, 5, 9, 2],
    row_lengths=[4, 0, 2, 1])

In [1]:
import numpy as np

np.zeros(shape=(3,16))

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])